In [1]:

import shutil
import subprocess
import os
from itertools import groupby
from operator import itemgetter
import numpy as np
import csv
import os
import time
from copy import copy
import pandas as pd
import pyAudioAnalysis
from pyAudioAnalysis import audioSegmentation as aS
from pyAudioAnalysis import audioBasicIO as aIO
import random
%matplotlib inline

os.chdir(os.path.expanduser('~/Dropbox/smacpy'))
from smacpy import Smacpy

random.seed(999)

In [2]:
#######################################
#### WAV path for testing purposes ####

speaker_value="Nixon, Richard"
audio_path="/Users/mclaugh/Desktop/AAPB_400_hrs_labeled/wav/cpb-aacip-75-88qbzvhx.h264.wav"
filename=audio_path.split('/')[-1][:-4]+'.mp4' ## kludge
print(filename)

#######################################

label_table_path="/Volumes/McLaughlin-6TB-1/Dropbox/aapb-data/AAPB_corpus_metadata/AAPB_speech_labels_full_corpus_170425.csv"
label_table = pd.read_csv(label_table_path)

cpb-aacip-75-88qbzvhx.h264.mp4


In [3]:
## Creating dictionary of label values for a given speaker.


speaker_range_dict={}

# extracting rows associated with specified speaker
speaker_rows = label_table[label_table['Value']==speaker_value]

# creating a list of all relevant filenames
speaker_filenames = sorted([item for item in list(set(list(speaker_rows['Filename']))) if str(item)!='nan'])

for filename in speaker_filenames:
    filename_rows = label_table[label_table['Filename']==filename]
    speaker_ranges=[]
    for index, row in filename_rows.iterrows():
        row = label_table.iloc[index]
        speaker_ranges.append((float(row['Timecode IN']),float(row['Timecode OUT'])))
    speaker_range_dict[filename]=speaker_ranges

#test
speaker_range_dict[filename]

[(16.0, 23.0),
 (1923.0, 1928.0),
 (1981.0, 2015.0),
 (2056.0, 2087.0),
 (2197.0, 2213.0),
 (2962.0, 2976.0)]

In [4]:
### Downsampling and applying bandpass filter ###

wav_path = '/var/tmp/'+filename+'.wav'

subprocess.call(['ffmpeg', '-y', '-i', audio_path, '-ar', '16000', '-ac', '1', '-af', "volume=0.99,highpass=f=150,lowpass=f=5500",wav_path])


## Full-quality version for the sake of feature extraction

wav_path_hifi = '/var/tmp/'+filename+'.hifi.wav'

subprocess.call(['ffmpeg', '-y', '-i', audio_path, wav_path_hifi])




In [ ]:
### Extracting formants with Praat ###

def extract_formants(wav_path):
    formant_lol = [line.split(',') for line in subprocess.check_output(["/Applications/Praat.app/Contents/MacOS/Praat", \
                "--run", "/Volumes/McLaughlin-6TB-1/Dropbox/smacpy/extract_formants.praat", \
                wav_path_hifi]).splitlines()]

    formant_table = pd.DataFrame(formant_lol[1:],columns=formant_lol[0])
    formant_table = formant_table.replace({'--undefined--':np.nan}) ## handling Praat's notation for undefined values

    formant_table['F1']=formant_table['F1'].astype(float,error='coerce')  ## Converting all values to float
    formant_table['F2']=formant_table['F2'].astype(float,error='coerce')
    formant_table['F3']=formant_table['F3'].astype(float,error='coerce')
    formant_table['F4']=formant_table['F4'].astype(float,error='coerce')
    formant_table['starttime']=formant_table['starttime'].astype(float,error='coerce')
    formant_table['endtime']=formant_table['endtime'].astype(float,error='coerce')

    for index, row in formant_table.iterrows():       ## Rounding start and end time values returned by Praat
        row=formant_table.iloc[index]
        row['starttime']=round(row['starttime'],3)
        row['endtime']=round(row['starttime'],3)

    return formant_table



#formant_table = extract_formants(wav_path_hifi)
#formant_table.head()

In [5]:
def extract_mfccs(wav_path):
    model = Smacpy("", {wav_path:'a'})
    return pd.DataFrame(model.file_to_features(wav_path))


#mfccs = extract_mfccs(wav_path)

Updating parameters and recalculating filters: 
('Nyquist: ', 8000.0)
minHz:0
maxHz:8000.0
minMel:0.0
maxMel:2840.06412216

Reading /var/tmp/cpb-aacip-75-88qbzvhx.h264.mp4.wav
    Training a GaussianMixture for label a, using data of shape (55375, 13)
  Trained 1 classes from 1 input files
Reading /var/tmp/cpb-aacip-75-88qbzvhx.h264.mp4.wav


In [ ]:

### list of seconds values like [1,2,9,10]
### to inclusive ranges like [(1,3),(9,11)]

def seconds_list_to_ranges(seconds_list): 
    ranges = []                
    for k, g in groupby(enumerate(seconds_list), lambda (i,x):i-x):
        group = map(itemgetter(1), g)
        ranges.append((group[0], group[-1]+1)) ## Adding 1 to make each range inclusive
    return ranges


### Hann smoothing

def smooth(x_in,window_len=10,window='hanning'):
        x=np.array(x_in)
        if x.ndim != 1:
                raise ValueError, "smooth only accepts 1 dimension arrays."
        if x.size < window_len:
                raise ValueError, "Input vector needs to be bigger than window size."
        if window_len<3:
                return x
        if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
                raise ValueError, "Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'"
        s=np.r_[2*x[0]-x[window_len-1::-1],x,2*x[-1]-x[-1:-window_len:-1]]
        if window == 'flat': #moving average
                w=np.ones(window_len,'d')
        else:  
                w=eval('np.'+window+'(window_len)')
        y=np.convolve(w/w.sum(),s,mode='same')
        return list(y[window_len:-window_len+1])


### Range intersection function cribbed from BallpointBen's comment here:
### https://stackoverflow.com/questions/40367461/intersection-of-two-lists-of-ranges-in-python


def intersects(a,b):
    ranges = []
    i = j = 0
    while i < len(a) and j < len(b):
        a_left, a_right = a[i]
        b_left, b_right = b[j]
        end_pts = sorted([a_left, a_right, b_left, b_right])
        middle = [end_pts[1], end_pts[2]]

        if a_right < b_right:
            i += 1
        else:
            j += 1

        if a_right >= b_left and b_right >= a_left:
            ranges.append(middle)

    for i in range(len(ranges)-1):
        if ranges[i][1] in (ranges[i+1][0], ranges[i+1][0]-1):
            ranges[i:i+2] = [[ranges[i][0], ranges[i+1][1]]]

    return ranges

#a = [[0,2.5], [5,9.9], [10,24.9]]
#b = [[1,5], [8,12], [15,18],[20,20.001], [21,24]]
#print(intersects(a,b))
# [[1, 2], [5, 5], [8, 10], [15, 18], [20, 24]]




In [ ]:
#pd.Series([float(item) for item in list(formant_table['F1'])]).plot()

In [ ]:
## Male-female classification

### 0 is male, 1 is female
### calculated at 1-second resolution


def find_male_ranges(wav_path):
    [mf_classes, classesAll, acc, CM] = aS.mtFileClassification(wav_path, "data/svmSpeakerFemaleMale", "svm", False)
    #classes = list(mf_classes) ## non-smoothed output
    smoothed_classes = [round(item) for item in smooth([int(item) for item in list(mf_classes)],window_len=7)] 
    counter=0
    male_secs=[]
    for segment in smoothed_classes:
        if segment < 1.0:
            male_secs.append(counter)
        counter+=1
    return seconds_list_to_ranges(male_secs)


## Female-male classification

### 0 is female, 1 is female
### calculated at 1-second resolution


def find_female_ranges(wav_path):
    [mf_classes, classesAll, acc, CM] = aS.mtFileClassification(wav_path, "data/svmSpeakerFemaleMale", "svm", False)
    #classes = list(mf_classes) ## non-smoothed output
    smoothed_classes = [round(item) for item in smooth([int(item) for item in list(mf_classes)],window_len=7)] 
    counter=0
    female_secs=[]
    for segment in smoothed_classes:
        if segment > 0.0:
            female_secs.append(counter)
        counter+=1
    return seconds_list_to_ranges(female_secs)


#male_ranges = find_male_ranges(wav_path)
#print(male_ranges)

#female_ranges = find_female_ranges(wav_path)
#print(female_ranges)

In [ ]:
## speech classification

### 0 is speech, 1 is music
### ['speech', 'music']

def find_non_music_ranges(wav_path):
    [classes, classesAll, acc, CM] = aS.mtFileClassification(wav_path, "data/svmSM", "svm", False)
    counter=0
    non_music_secs=[]
    for segment in classes:
        if segment < 1.0:
            non_music_secs.append(counter)
        counter+=1
    return seconds_list_to_ranges(non_music_secs)


In [ ]:
non_music_ranges = find_non_music_ranges(wav_path_hifi)

print(non_music_ranges)

In [ ]:
## applause classification

### 0 is non-applause, 1 is applause

def find_non_applause_ranges(wav_path):
    [classes, classesAll, acc, CM] = aS.mtFileClassification(wav_path, "data/svm_applause_model", "svm", False)
    counter=0
    non_applause_secs=[]
    for segment in classes:
        if segment < 1.0:
            non_applause_secs.append(counter)
        counter+=1
    return seconds_list_to_ranges(non_applause_secs)


In [ ]:
non_applause_ranges = find_non_applause_ranges(wav_path_hifi)

print(non_applause_ranges)

In [ ]:
def find_non_silence_ranges(wav_path):
    [Fs, x] = aIO.readAudioFile(wav_path)
    segments = aS.silenceRemoval(x, Fs, 0.020, 0.020, smoothWindow = 1.0, Weight = 0.3, plot = False)
    return segments


In [ ]:
non_silence_ranges = find_non_silence_ranges(wav_path_hifi)

non_silence_ranges

In [ ]:
speech_ranges = intersects(non_applause_ranges, intersects(non_music_ranges,non_silence_ranges)) ## intersecting 3 range lists

speech_ranges

In [ ]:
speaker_range_dict[filename]

In [ ]:
intersects(speech_ranges,speaker_range_dict[filename])

In [ ]:
#############################
## Enter Speaker Name Here ##
#############################

speaker = "Clinton, Hillary"

corpus_dir = "/Volumes/McLaughlin-6TB-1/Dropbox/aapb-hipstas/Model_training_clips/"

#############################

last_name = speaker.split(', ')[0]

print(speaker)
print(last_name)

In [ ]:

ubm_dir = os.path.join(corpus_dir,'UBM_'+last_name)


speaker_dir =os.path.join(corpus_dir,speaker)

num_speaker_clips=len([item for item in os.listdir(speaker_dir) if item.lower()[-4:]=='.wav'])

smacpy_dict={}

for filename in [item for item in os.listdir(ubm_dir) if '.wav' in item]:
    smacpy_dict[os.path.join(ubm_dir,filename)]="background"

for filename in [item for item in os.listdir(speaker_dir) if '.wav' in item]:
    smacpy_dict[os.path.join(speaker_dir,filename)]=speaker_dir.strip('/').split('/')[-1]

model = Smacpy("", smacpy_dict)

In [ ]:

def classify_audio_file(audio_pathname):
    segments_dir_temp = os.path.join(corpus_dir, last_name+'_segments_1_sec')
    basename = audio_pathname.split('/')[-1][:-4]
    try:
        os.mkdir(segments_dir_temp)
        subprocess.call(['ffmpeg','-i',audio_pathname,'-n','-f','segment','-segment_time','1','-c','copy',os.path.join(segments_dir_temp,basename+"_sec_%05d.wav")])
    except:
        print("Apparently already processed: "+audio_pathname)
    seg_pathnames = [os.path.join(segments_dir_temp, item) for item in os.listdir(segments_dir_temp) if item.lower()[-4:]=='.wav']
    output=[]
    global speaker
    for filename in [item for item in seg_pathnames if '.wav' in item]:
        if model.classify(filename) == speaker:
            output.append(1)
        else:
            output.append(0)
    
    counter=0
    speaker_secs=[]
    for segment in output:
        if segment>0.0:
            speaker_secs.append(counter)
        counter+=1
    speaker_ranges = seconds_list_to_ranges(speaker_secs)
    csv_pathname = audio_pathname[:-4]+'_GMM_UBM_'+last_name+'_'+str(num_speaker_clips)+"x2s.csv"
    with open(csv_pathname, 'w') as csv_fo:
        speaker_ranges_expanded=[(start,1,end-start+1) for start,end in speaker_ranges]
        csv_writer = csv.writer(csv_fo)
        csv_writer.writerows(speaker_ranges_expanded)
    ## Smooth version
    output_smooth_temp = list(smooth(np.array(output),window_len=10))
    output_smooth=[]
    for item in output_smooth_temp:
        output_smooth.append(round(item))
    counter=0
    speaker_secs=[]
    for segment in output_smooth:
        if segment>0.0:
            speaker_secs.append(counter)
        counter+=1
    speaker_ranges = seconds_list_to_ranges(speaker_secs)
    csv_smooth_pathname = audio_pathname[:-4]+'_GMM_UBM_smooth10_'+last_name+'_'+str(num_speaker_clips)+"x2s.csv"
    with open(csv_smooth_pathname, 'w') as csv_fo:
        speaker_ranges_expanded=[(start,1,end-start+1) for start,end in speaker_ranges]
        csv_writer = csv.writer(csv_fo)
        csv_writer.writerows(speaker_ranges_expanded)
    shutil.rmtree(segments_dir_temp)


    


In [ ]:

test_set_dir="/Volumes/U/AAPB_Corpus_May_2017/Test_Sets/AAPB_Test_Haystack_"+last_name

to_classify=[os.path.join(test_set_dir,item) for item in os.listdir(test_set_dir) if ('.wav' in item)&(item[0]!='.')]

for pathname in to_classify:
    classify_audio_file(pathname)